In [2]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
        
    return str_text

In [4]:
#read_file('moby_dick_four_chapters.txt')

In [5]:
import spacy 

In [6]:
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

In [7]:
nlp.max_length = 1198623

In [8]:
def seperate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [9]:
d = read_file('moby_dick_four_chapters.txt')

In [10]:
tokens = seperate_punc(d)

In [13]:
len(tokens)

11394

In [14]:
# 25 words --> network predict #26

In [20]:
train_len = 25+1
text_sequences = []

for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]
    
    text_sequences.append(seq)

In [21]:
type(text_sequences)

list

In [24]:
' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [25]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [26]:
from keras.preprocessing.text import Tokenizer

In [28]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [29]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [32]:
#sequences

In [35]:
for i in sequences[0]:
    print(f"{i}: {tokenizer.index_word[i]}")

964: call
14: me
265: ishmael
51: some
263: years
416: ago
87: never
222: mind
129: how
111: long
962: precisely
262: having
50: little
43: or
37: no
321: money
7: in
23: my
555: purse
3: and
150: nothing
261: particular
6: to
2704: interest
14: me
24: on


In [37]:
#tokenizer.word_counts

In [38]:
vocabulary_size = len(tokenizer.word_counts)

In [39]:
vocabulary_size

2709

In [41]:
#tokenizer.index_word

In [45]:
#sequences[0]

In [46]:
import numpy as np

In [47]:
sequences = np.array(sequences)

In [48]:
sequences

array([[ 964,   14,  265, ..., 2704,   14,   24],
       [  14,  265,   51, ...,   14,   24,  965],
       [ 265,   51,  263, ...,   24,  965,    5],
       ...,
       [ 960,   12,  168, ...,  264,   53,    2],
       [  12,  168, 2703, ...,   53,    2, 2709],
       [ 168, 2703,    3, ...,    2, 2709,   26]])

In [54]:
from keras.utils import to_categorical

In [55]:
X = sequences[:,:-1]

In [56]:
y = sequences[:,-1]

In [57]:
y = to_categorical(y, num_classes=vocabulary_size+1)

In [60]:
seq_len = X.shape[1]

In [62]:
X.shape

(11368, 25)

In [63]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [64]:
def create_model(vocabulary_size, seq_len):
    
    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50, activation='relu'))
    
    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
    
    return model

In [65]:
model = create_model(vocabulary_size+1, seq_len)

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            67750     
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 50)            15200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 2710)              138210    
Total params: 243,910
Trainable params: 243,910
Non-trainable params: 0
_________________________________________________________________


In [66]:
from pickle import dump,load

In [67]:
model.fit(X, y, batch_size=128, epochs=2, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
11368/11368 [==============================] - 36s 3ms/step - loss: 6.9716 - accuracy: 0.0342
Epoch 2/2
11368/11368 [==============================] - 24s 2ms/step - loss: 6.3771 - accuracy: 0.0527


In [68]:
model.save('my_mobydick_model.h5')

In [70]:
dump(tokenizer, open('my_simpletokenizer','wb'))

In [72]:
from keras.preprocessing.sequence import pad_sequences

In [105]:
def generate_text(model, tokenizer, seq_len, seed_text,num_gen_words):
    
    output_text =[]
    
    # 25 words
    input_text = seed_text
    
    for i in range(num_gen_words):
        
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        pred_word_ind = model.predict_classes(pad_encoded, verbose = 0)[0]
        pred_word = tokenizer.index_word[pred_word_ind]
        input_text += ' '+pred_word
        
        output_text.append(pred_word)
    
    return ' '.join(output_text)
    

In [106]:
text_sequences[0]

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on']

In [107]:
import random
random.seed(101)
random_pick = random.randint(0, len(text_sequences))

In [108]:
random_seed_text = text_sequences[random_pick]

In [109]:
random_seed_text

['and',
 'throwing',
 'the',
 'clothes',
 'to',
 'one',
 'side',
 'he',
 'really',
 'did',
 'this',
 'in',
 'not',
 'only',
 'a',
 'civil',
 'but',
 'a',
 'really',
 'kind',
 'and',
 'charitable',
 'way',
 'i',
 'stood',
 'looking']

In [110]:
seed_text = ' '.join(random_seed_text)

In [111]:
seed_text

'and throwing the clothes to one side he really did this in not only a civil but a really kind and charitable way i stood looking'

In [112]:
generate_text(model, tokenizer, seq_len, seed_text = seed_text, num_gen_words=25)

'the the the the the the the the the the the the the the the the the the the the the the the the the'

In [113]:
from keras.models import load_model

In [114]:
model = load_model('epochBIG.h5')

In [115]:
tokenizer = load(open('epochBIG', 'rb'))

In [116]:
generate_text(model, tokenizer, seq_len, seed_text = seed_text, num_gen_words=25)

"at that stubb ' my frame roman eyes of his own power for the whale 's grain to wrenched progeny for a fever drawn up"